In [1]:
#allows to import generator and discriminator
!pip install -q git+https://github.com/tensorflow/examples.git

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import tensorflow as tf
#import tensorflow_datasets as tfds
from tensorflow_examples.models.pix2pix import pix2pix
from os import listdir
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from numpy import vstack
from numpy import asarray
from numpy import savez_compressed
import numpy as np
from PIL import Image
from tensorflow.keras.utils import plot_model

import os
import time
import matplotlib.pyplot as plt
from IPython.display import clear_output

#AUTOTUNE = tf.data.AUTOTUNE
AUTOTUNE = tf.data.experimental.AUTOTUNE

from PIL import Image
import glob

In [3]:
import tensorflow as tf
tf.version.VERSION

'2.7.0'

# Useful methods

In [4]:
# load all images in a directory into memory
def load_images(path, size=(256,256)):
    data_list = list()
    #enumerate filenames in directory, assume all are images
    for filename in listdir(path):
        # load and resize the image
        pixels = load_img(path + filename, target_size=size)
        # convert to numpy array
        pixels = img_to_array(pixels)
        # store
        data_list.append(pixels)
    return asarray(data_list)

In [5]:
def generate_images(model, test_input, c, month, folder, label):
    
    pair_path = '../imgs_results/wl2nbi/fake_images_ck4/igho/' + month + '/' + folder + '/pair_images/'
    if not os.path.exists(pair_path):
        os.makedirs(pair_path)
    
    wl_path = '../imgs_results/wl2nbi/fake_images_ck4/igho/' + month + '/' + folder + '/realWL/'
    if not os.path.exists(wl_path):
        os.makedirs(wl_path)
    
    artif_path = '../imgs_results/wl2nbi/fake_images_ck4/igho/' + month + '/' + folder + '/artifNBI/'
    if not os.path.exists(artif_path):
        os.makedirs(artif_path)
    
    
    prediction = model(test_input)
    display_list = [test_input[0], prediction[0]]
    
    plt.figure(figsize=(12, 12))
    
    #saving singles images
    
    #real WL
    name_file = wl_path + label + '_img_' + str(c) + '.png'
    # getting the pixel values between [0, 1] to plot it.
    tf.keras.preprocessing.image.save_img(name_file, display_list[0] * 0.5 + 0.5)
    
    
    #artif NBI
    name_file = artif_path + label + '_img_' + str(c) + '.png'
    # getting the pixel values between [0, 1] to plot it.
    tf.keras.preprocessing.image.save_img(name_file, display_list[1] * 0.5 + 0.5)

    title = ['Input Image', 'Predicted Image']
        
    for i in range(2):
        plt.subplot(1, 2, i+1)
        plt.title(title[i])
        # getting the pixel values between [0, 1] to plot it.
        plt.imshow(display_list[i] * 0.5 + 0.5)
        plt.axis('off')
        
    name_file = pair_path + label + '_img_' + str(c) + '.png'
    plt.savefig(name_file, format='png')
    plt.show()

**Data augmentation techniques**

In [6]:
def random_crop(image):
    cropped_image = tf.image.random_crop(
      image, size=[IMG_HEIGHT, IMG_WIDTH, 3])

    return cropped_image

# scaling the images to [-1, 1]
def normalize(image):
    image = tf.cast(image, tf.float32)
    image = (image / 127.5) - 1
    return image

def random_jitter(image):
    # resizing to 286 x 286 x 3
    image = tf.image.resize(image, [286, 286],
                          method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

    # randomly cropping to 256 x 256 x 3
    image = random_crop(image)

    # random mirroring
    image = tf.image.random_flip_left_right(image)

    return image

**Preprocess splits**

In [7]:
def preprocess_image_train(image):
    image = random_jitter(image)
    image = normalize(image)
    return image

def preprocess_image_test(image):
    image = normalize(image)
    return image

**Import and reuse the Pix2Pix models**

In [8]:
OUTPUT_CHANNELS = 3

generator_g = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')
generator_f = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')

discriminator_x = pix2pix.discriminator(norm_type='instancenorm', target=False)
discriminator_y = pix2pix.discriminator(norm_type='instancenorm', target=False)

**Initializing optimizers, generator and discriminators**

In [9]:
generator_g_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
generator_f_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

discriminator_x_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_y_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

# Loading generator model

In [10]:
checkpoint_path = "../models/rgb/"
ckpt = tf.train.Checkpoint(generator_g=generator_g,
                           generator_f=generator_f,
                           discriminator_x=discriminator_x,
                           discriminator_y=discriminator_y,
                           generator_g_optimizer=generator_g_optimizer,
                           generator_f_optimizer=generator_f_optimizer,
                           discriminator_x_optimizer=discriminator_x_optimizer,
                           discriminator_y_optimizer=discriminator_y_optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print ('Latest checkpoint restored!!')

Latest checkpoint restored!!


In [11]:
ckpt.restore(ckpt_manager.latest_checkpoint)
if ckpt_manager.latest_checkpoint:
    print("Restored from {}".format(ckpt_manager.latest_checkpoint))
else:
    print("Initializing from scratch.")

Restored from ../models/rgb/ckpt-12


In [ ]:
generator_g.summary()

In [ ]:
plot_model(generator_g, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [12]:
BUFFER_SIZE = 1000
BATCH_SIZE = 1
IMG_WIDTH = 256
IMG_HEIGHT = 256

In [ ]:
# frames from video path
path = "../../../../pregrado/data/RGB/WL/adenoma_WL/video_1/"
# load dataset white light
# here A: white light, B: nbi light
adenoma_WL = load_images(path)
print("Adenoma WL video_1: ", adenoma_WL.shape)

In [13]:
adenoma_WL_array = np.asarray(adenoma_WL)
adenoma_WL_ds = tf.data.Dataset.from_tensor_slices(adenoma_WL_array)
adenoma_WL_ds = adenoma_WL_ds.map(preprocess_image_test, num_parallel_calls=AUTOTUNE).cache().shuffle(
                BUFFER_SIZE).batch(BATCH_SIZE)

NameError: name 'adenoma_WL' is not defined

## **testeando**

In [ ]:
# Run the trained model on the test dataset
for inp in adenoma_WL_ds.take(adenoma_WL.shape[0]):
    generate_images(generator_g, inp)

# IGHO predicting
Nota: septiembre esta en proceso

In [24]:
month = "SEPTIEMBRE"
txt_folder = '../imgs_results/wl2nbi/fake_images_ck4/igho/' + month + '/' + 'folders.txt'
with open(txt_folder) as f:
    content = f.readlines()
    
folders = [cont.split('\n')[0] for cont in content]
print(folders)

['2021-09-02-106236', '2021-09-02-106239', '2021-09-02-106262', '2021-09-09-106762', '2021-09-09-106786', '2021-09-11-107014', '2021-09-11-107050']


In [25]:

folder = folders[0]
label = 'adenoma'
#test
txt_path = '../imgs_results/wl2nbi/fake_images_ck4/igho/' + month + '/' + folder + '/' + 'polyps.txt'

pathImgs = "/data/Datasets/Igho/Uncompressed/Videos/"+ month + "/" + folder + "/Normal/"

with open(txt_path) as f:
    content = f.readlines()

frames = [cont.split('.')[0] for cont in content]

data_list = list()
size=(256,256)
for frame in frames:
    to_read = pathImgs + frame + '.png'
    pixels = load_img(to_read, target_size=size)
    # convert to numpy array
    pixels = img_to_array(pixels)
    # store
    data_list.append(pixels)

WL = asarray(data_list)

print("The folder: {} has a shape of: {}".format(folder,WL.shape))

FileNotFoundError: [Errno 2] No such file or directory: '../imgs_results/wl2nbi/fake_images_ck4/igho/SEPTIEMBRE/2021-09-02-106236/polyps.txt'

In [ ]:
WL_array = np.asarray(WL)
WL_ds = tf.data.Dataset.from_tensor_slices(WL_array)
WL_ds = WL_ds.map(preprocess_image_test, num_parallel_calls=AUTOTUNE).cache().shuffle(
                BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
for index, inp in zip(frames, WL_ds.take(WL.shape[0])):
    generate_images(generator_g, inp, index, month, folder, label)

# Predicting over full frames videos

In [ ]:
def generatingImages(data, generator, salve_path):
    
    for count, inp in enumerate(data):
        fake = generator(inp)
        fake = fake[0]* 0.5 + 0.5
        #para que PIL Image deje guardar (mult por 255 and change by uint8)
        fake = np.array(fake) * 255
        fake = fake.astype(np.uint8)
        fake_img = Image.fromarray(fake)
        
        #checking if directory exist
        directory = salve_path.split('/')[:-1]
        directory = "/".join(directory)
        if not os.path.exists(directory):
            os.mkdir(directory)
        
        clase = directory.split('/')[-2:]
        clase = "_".join(clase)
        salve_path = directory + '/' + clase + '_img_' + str(count) + '.png'
        fake_img.save(salve_path)    

In [ ]:
def toDataSet(path_origen):
    data = load_images(path_origen)
    print("dimensiones de data: ", data.shape)
    data_array = np.asarray(data)
    data_ds = tf.data.Dataset.from_tensor_slices(data_array)
    data_ds = data_ds.map(preprocess_image_test, num_parallel_calls=AUTOTUNE).cache().shuffle(
                    BUFFER_SIZE).batch(BATCH_SIZE)
    
    return data_ds

In [ ]:
# frames from video path
path = '/Users/frank/Documents/Tesis/Polipos/DataSet/PolypDatasetProcesed/WL/'
save = '/Users/frank/Documents/maestria/propuesta/experiments/data/wl2nbi/fake_images_ck80/'
clases = os.listdir(path)
for clase in clases:
    print("========= clase ", clase, "=========")
    clase_pth = path + clase + '/'
    videos = os.listdir(clase_pth)
    for video in videos:
        print("===",video, "====")
        video_pth = clase_pth + video + '/'
        print("convirtiendo a tf.Dataset...")
        data_ds = toDataSet(video_pth)
        imgs = os.listdir(video_pth)
        print("prediciendo con cycleGan...")
        path_save = save + clase + '/' + video + '/'
        generatingImages(data_ds, generator_g, path_save)
            

print("Finalizado!")

# Creating video from fake frames predicted

In [ ]:
import cv2
import numpy as np
import glob

path = '/Users/frank/Documents/maestria/propuesta/experiments/data/wl2nbi/fake_images_ck80/serrated_WL/video_2/*.png'

img_array = []
for filename in glob.glob(path):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)

out = cv2.VideoWriter('serV2.avi',cv2.VideoWriter_fourcc(*'DIVX'), 7, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()